In [87]:
from torchvision.datasets import MNIST
import torchvision
import numpy as np
from torchvision.transforms import ToPILImage
to_img = ToPILImage()
from PIL import Image
from torch.autograd import Variable
import torch
import torch.nn as nn

In [88]:
class View (nn.Module):
    def __init__(self, size):
        super(View, self).__init__()
        self.size=size
        
    def forward(self, tensor):
        return tensor.view(self.size)

In [89]:
def reparametrize(mu, logvar):
    std=logvar.div(2).exp()
    eps=Variable(std.data.new(std.size()).normal_())
    return mu+std*eps


class BetaVAE(nn.Module):
    
    def __init__(self, z_dim=10):
        super(BetaVAE, self).__init__()
        self.z_dim=z_dim
        
        self.encoder=nn.Sequential(
        nn.Conv2d(1, 32, 5, 1),
        nn.ReLU(True),
        nn.Conv2d(32,32, 5, 1),
        nn.ReLU(True),
        nn.Conv2d(32, 32, 4, 2),
        nn.ReLU(True),
        nn.Conv2d(32, 64, 3, 2),
        nn.ReLU(True),
        nn.Conv2d(64, 256, 4, 1),
        nn.ReLU(True),
        View((-1, 256*1*1)),
        nn.Linear(256, z_dim*2))
        
        self.decoder=nn.Sequential(
        nn.Linear(z_dim, 256),
        View((-1, 256, 1, 1)),
        nn.ReLU(True),
        nn.ConvTranspose2d(256, 64, 4, 1),
        nn.ReLU(True),
        nn.ConvTranspose2d(64, 32, 3, 2),
        nn.ReLU(True),
        nn.ConvTranspose2d(32, 32, 4, 2),
        nn.ReLU(True),
        nn.ConvTranspose2d(32, 32, 5, 1),
        nn.ReLU(True),
        nn.ConvTranspose2d(32, 1, 5, 1))
        
        
    def forward(self, x):
        distributions=self.encoder(x)
     
        mu=distributions[:, :self.z_dim]
        logvar=distributions[:, self.z_dim:]
     
        z=reparametrize(mu, logvar)
        x_recon=self.decoder(z)
        
        return x_recon, mu, logvar
        
        

In [117]:
transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset=MNIST(root="~/torch_datasets", train=True, transform=transform, download=True)

test_dataset=MNIST(root="~/torch_datasets", train=False, transform=transform, download=True)


train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)

test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
        

In [91]:
model=BetaVAE(32)
criterion=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
num_epochs=30


In [118]:
idx=(train_dataset.targets==1)|(train_dataset.targets==3)|(train_dataset.targets==5)

In [119]:
idx

tensor([ True, False, False,  ...,  True, False, False])

In [120]:
train_dataset.data=train_dataset.data[idx]


In [121]:
train_dataset.targets=train_dataset.targets[idx]

In [122]:
len(train_dataset.data)

18294

In [123]:
idx_ts=(test_dataset.targets==1)|(test_dataset.targets==3)|(test_dataset.targets==5)

In [124]:
test_dataset.data=test_dataset.data[idx_ts]

In [125]:
test_dataset.targets=test_dataset.targets[idx_ts]

In [126]:
len(test_dataset.data)

3037

In [127]:
for epoch in range (num_epochs):
    for data in train_loader:
        model.train()
        inp, _ = data
        output, mu, logvar=model(inp)
        kl = -0.5 * (1 + logvar - mu.pow(2)- logvar.exp())
        kl_loss=kl.sum()/inp.shape[0]
        
        std=torch.exp(0.5*logvar)
        z=torch.empty_like(mu).normal_()*std+mu
        
        fit=0.5*(output-inp).pow(2)
        fit_loss=fit.sum()/inp.shape[0]
        
        loss= kl_loss + fit_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))
       

epoch [1/30], loss:15.0626
epoch [2/30], loss:15.0798
epoch [3/30], loss:14.4823
epoch [4/30], loss:13.6135
epoch [5/30], loss:14.5496
epoch [6/30], loss:14.0098
epoch [7/30], loss:14.8292
epoch [8/30], loss:14.8475
epoch [9/30], loss:14.4044
epoch [10/30], loss:14.8528
epoch [11/30], loss:13.8983
epoch [12/30], loss:14.5446
epoch [13/30], loss:14.4263
epoch [14/30], loss:14.6838
epoch [15/30], loss:13.5833
epoch [16/30], loss:14.9641
epoch [17/30], loss:14.6028
epoch [18/30], loss:14.5849
epoch [19/30], loss:14.8681
epoch [20/30], loss:14.4346
epoch [21/30], loss:15.2463
epoch [22/30], loss:14.9727
epoch [23/30], loss:14.7063
epoch [24/30], loss:14.2169
epoch [25/30], loss:14.9502
epoch [26/30], loss:13.4605
epoch [27/30], loss:15.0935
epoch [28/30], loss:15.1881
epoch [29/30], loss:14.6031
epoch [30/30], loss:14.0429


In [128]:
for data in test_loader:
        model.eval()
        inp, _ = data
        output, mu, logvar=model(inp)
        kl = -0.5 * (1 + logvar - mu.pow(2)- logvar.exp())
        kl_loss=kl.sum()/inp.shape[0]
        
        std=torch.exp(0.5*logvar)
        z=torch.empty_like(mu).normal_()*std+mu
        
        fit=0.5*(output-inp).pow(2)
        fit_loss=fit.sum()/inp.shape[0]
        
        loss= kl_loss + fit_loss
print('loss:{:.4f}'.format(loss))
        
        

loss:14.5721


In [129]:
torch.save(model.state_dict(), './var_autoencoder02.pth')

In [138]:
y=test_dataset[89][0]

In [139]:
to_img(y)

In [140]:
x=model(y.reshape((1, -1,28, 28 )))

In [141]:
to_img(x[0].reshape(1, 28, 28))